In [57]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 

In [83]:
flare_list = pd.read_csv("/Users/laurahayes/ml_project_flares/flare_analysis/goes_flare_list/swpc_event_list.csv")
flare_list["matchtime"] = pd.to_datetime(flare_list["ts"], format="%Y%m%d%H%M").dt.strftime("%Y-%m-%d 00:30")
flare_list["noaa_ar"] = flare_list["noaa_ar"].astype(str)


flare_list["event_starttime"] = pd.to_datetime(flare_list["date"].astype(str) + flare_list["start_time"].astype(str).str.zfill(4))
flare_list["event_peaktime"] = pd.to_datetime(flare_list["date"].astype(str) + flare_list["max_time"].astype(str).str.zfill(4))
flare_list["event_endtime"] = pd.to_datetime(flare_list["date"].astype(str) + flare_list["end_time"].astype(str).str.zfill(4))

flare_list.drop(["start_time", "max_time", "end_time", "event_no", "ts"], axis=1, inplace=True)

In [84]:
flare_list.head(3)

,date,goes_sat,goes_channel,goes_class_ind,goes_class,integrated_flux,noaa_ar,matchtime,event_starttime,event_peaktime,event_endtime
0,20100102,G14,1-8A,C,C1.0,1.2E-03,11039,2010-01-02 00:30,2010-01-02 07:09:00,2010-01-02 07:24:00,2010-01-02 07:40:00
1,20100102,G14,1-8A,C,C2.6,1.4E-03,,2010-01-02 00:30,2010-01-02 14:08:00,2010-01-02 14:16:00,2010-01-02 14:24:00
2,20100102,G14,1-8A,C,C3.1,4.5E-03,11039,2010-01-02 00:30,2010-01-02 22:57:00,2010-01-02 23:19:00,2010-01-02 23:35:00


In [85]:
flare_list["noaa_ar"].astype(str)

0       11039
1            
2       11039
3       11039
4       11039
        ...  
8601    12712
8602    12712
8603    12712
8604    12715
8605         
Name: noaa_ar, Length: 8606, dtype: object

In [86]:
flare_list["goes_class_ind"].value_counts()

C    7809
M     748
X      49
Name: goes_class_ind, dtype: int64

In [87]:
ar_data = pd.read_csv("/Users/laurahayes/ml_project_flares/flare_analysis/AR_analysis/SRS_all_2010-2018.csv")
ar_data = ar_data[ar_data["ID"].isin(["I"])]
ar_data["matchtime"] = ar_data["date"]
ar_data["noaa_ar"] = ar_data["Number"].astype(str)
ar_data.rename(columns={"date":"AR issue_date"}, inplace=True)

In [88]:
ar_data.head(3)

,ID,Number,Carrington Longitude,Area,Z,Longitudinal Extent,Number of Sunspots,Mag Type,Latitude,Longitude,AR issue_date,matchtime,noaa_ar
0,I,11039,54.0,130.0,Dai,6.0,8.0,Beta,-28.0,21.0,2010-01-01 00:30,2010-01-01 00:30,11039
1,I,11039,54.0,120.0,Dso,9.0,6.0,Beta,-27.0,35.0,2010-01-02 00:30,2010-01-02 00:30,11039
3,I,11039,52.0,190.0,Dso,8.0,12.0,Beta,-28.0,46.0,2010-01-03 00:30,2010-01-03 00:30,11039


In [89]:
flare_db = pd.merge(flare_list, ar_data, how="left", on=["matchtime", "noaa_ar"])

In [90]:
flare_db

,date,goes_sat,goes_channel,goes_class_ind,goes_class,integrated_flux,noaa_ar,matchtime,event_starttime,event_peaktime,...,Number,Carrington Longitude,Area,Z,Longitudinal Extent,Number of Sunspots,Mag Type,Latitude,Longitude,AR issue_date
0,20100102,G14,1-8A,C,C1.0,1.2E-03,11039,2010-01-02 00:30,2010-01-02 07:09:00,2010-01-02 07:24:00,...,11039.0,54.0,120.0,Dso,9.0,6.0,Beta,-27.0,35.0,2010-01-02 00:30
1,20100102,G14,1-8A,C,C2.6,1.4E-03,,2010-01-02 00:30,2010-01-02 14:08:00,2010-01-02 14:16:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20100102,G14,1-8A,C,C3.1,4.5E-03,11039,2010-01-02 00:30,2010-01-02 22:57:00,2010-01-02 23:19:00,...,11039.0,54.0,120.0,Dso,9.0,6.0,Beta,-27.0,35.0,2010-01-02 00:30
3,20100103,G14,1-8A,C,C2.0,1.1E-03,11039,2010-01-03 00:30,2010-01-03 01:17:00,2010-01-03 01:22:00,...,11039.0,52.0,190.0,Dso,8.0,12.0,Beta,-28.0,46.0,2010-01-03 00:30
4,20100104,G14,1-8A,C,C1.3,2.1E-04,11039,2010-01-04 00:30,2010-01-04 03:36:00,2010-01-04 03:40:00,...,11039.0,52.0,220.0,Dso,7.0,10.0,Beta,-29.0,59.0,2010-01-04 00:30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8601,20180523,G15,1-8A,C,C2.0,2.4E-03,12712,2018-05-23 00:30,2018-05-23 18:03:00,2018-05-23 18:21:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8602,20180528,G15,1-8A,C,C2.7,4.4E-04,12712,2018-05-28 00:30,2018-05-28 17:04:00,2018-05-28 17:10:00,...,12712.0,179.0,40.0,Cro,9.0,5.0,Beta,15.0,-28.0,2018-05-28 00:30
8603,20180606,G15,1-8A,C,C1.0,6.6E-04,12712,2018-06-06 00:30,2018-06-06 10:44:00,2018-06-06 11:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8604,20180621,G15,1-8A,C,C2.1,6.4E-04,12715,2018-06-21 00:30,2018-06-21 01:09:00,2018-06-21 01:15:00,...,12715.0,230.0,110.0,Dai,6.0,12.0,Beta,8.0,-20.0,2018-06-21 00:30


In [91]:
flare_list.shape

(8606, 11)